<a href="https://colab.research.google.com/github/neo4j/graph-data-science-client/blob/main/examples/load-data-via-graph-construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data to a projected graph via graph construction

This notebook shows the usage of the `gds.alpha.graph.construct` method (available only in GDS 2.1+) to build a graph directly in memory.

**NOTE:** If you are using AuraDS, it is currently not possible to write the projected graph back to Neo4j.

## Setup

We need an environment where Neo4j and GDS are available, for example AuraDS (which comes with GDS preinstalled) or Neo4j Desktop. 

Once the credentials to this environment are available, we can install the `graphdatascience` package and import the client class.

In [ ]:
!pip install graphdatascience

In [20]:
from graphdatascience import GraphDataScience

When using a local Neo4j setup, the default connection URI is `bolt://localhost:7687`:

In [ ]:
# Replace with the actual connection URI and credentials
NEO4J_CONNECTION_URI = "bolt://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = ""

gds = GraphDataScience(NEO4J_CONNECTION_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

When using AuraDS, the connection URI is slightly different as it uses the `neo4j+s` protocol. The client should also include the `aura_ds=True` flag to enable AuraDS-recommended settings.


In [ ]:
# Replace with the actual connection URI and credentials
NEO4J_CONNECTION_URI = "neo4j+s://xxxxxxxx.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = ""

gds = GraphDataScience(NEO4J_CONNECTION_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), aura_ds=True)

We also import `pandas` to create a Pandas `DataFrame` from the original data source.

In [25]:
import pandas as pd

## Load the Cora dataset

In [26]:
CORA_CONTENT = (
    "https://raw.githubusercontent.com/neo4j/graph-data-science/master/test-utils/src/main/resources/cora.content"
)
CORA_CITES = (
    "https://raw.githubusercontent.com/neo4j/graph-data-science/master/test-utils/src/main/resources/cora.cites"
)

We can load each CSV locally as a Pandas `DataFrame`.

In [27]:
content = pd.read_csv(CORA_CONTENT, header=None)
cites = pd.read_csv(CORA_CITES, header=None)

We need to perform an additional preprocessing step to convert the `subject` field (which is a string in the dataset) into an integer, because node properties have to be numerical in order to be projected into a graph. We can use a map for this.

In [28]:
SUBJECT_TO_ID = {
    "Neural_Networks": 0,
    "Rule_Learning": 1,
    "Reinforcement_Learning": 2,
    "Probabilistic_Methods": 3,
    "Theory": 4,
    "Genetic_Algorithms": 5,
    "Case_Based": 6,
}

We can now create a new `DataFrame` with a `nodeId` field, a list of node labels,
and the additional node properties `subject` (using the `SUBJECT_TO_ID` 
mapping) and `features` (converting all the feature columns to a single
array column).

In [29]:
nodes = pd.DataFrame().assign(
    nodeId=content[0],
    labels="Paper",
    subject=content[1].replace(SUBJECT_TO_ID),
    features=content.iloc[:, 2:].apply(list, axis=1),
)

Let's check the first 5 rows of the new `DataFrame`:

In [32]:
nodes.head()

,nodeId,labels,subject,features
0,31336,Paper,0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1061127,Paper,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
2,1106406,Paper,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,13195,Paper,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,37879,Paper,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


Now we create a new `DataFrame` containing the relationships between the nodes.
To create the equivalent of an undirected graph, we need to add direct
and inverse relationships explicitly.

In [33]:
dir_relationships = pd.DataFrame().assign(sourceNodeId=cites[0], targetNodeId=cites[1], relationshipType="CITES")
inv_relationships = pd.DataFrame().assign(sourceNodeId=cites[1], targetNodeId=cites[0], relationshipType="CITES")

relationships = pd.concat([dir_relationships, inv_relationships]).drop_duplicates()

Again, let's check the first 5 rows of the new `DataFrame`:

In [34]:
relationships.head()

,sourceNodeId,targetNodeId,relationshipType
0,35,1033,CITES
1,35,103482,CITES
2,35,103515,CITES
3,35,1050679,CITES
4,35,1103960,CITES


Finally, we can create the in-memory graph.

In [ ]:
G = gds.alpha.graph.construct("cora-graph", nodes, relationships)

## Use the graph

Let's check that the new graph has been created:

In [ ]:
gds.graph.list()

Let's also count the nodes in the graph:

In [40]:
G.node_count()

2708

The count matches with the number of rows in the Pandas dataset:

In [42]:
len(content)

2708

We can stream the value of the `subject` node property for
each node in the graph, printing only the first 10.

In [43]:
gds.graph.streamNodeProperties(G, ["subject"]).head(10)

,nodeId,nodeProperty,propertyValue
0,31336,subject,0
1,1061127,subject,1
2,1106406,subject,2
3,13195,subject,2
4,37879,subject,3
5,1126012,subject,3
6,1107140,subject,4
7,1102850,subject,0
8,31349,subject,0
9,1106418,subject,4


## Cleanup

When the graph is no longer needed, it should be dropped to free up memory:

In [ ]:
G.drop()